In [ ]:
import re
from datetime import datetime
#import pillow
import os
import rasterio
import matplotlib.pyplot as plt
import numpy as np

In [ ]:

def extract_date(filename):
    # Regular expression to extract the date part from the filename
    match = re.search(r'(\d{7})T', filename)
    if match:
        date_str = match.group(1)
        # Convert to a datetime object
        date = datetime.strptime(date_str, '%Y%j')
        return date
    return None

def sort_filenames_by_date(filenames):

    date_filename_pairs = [(extract_date(filename), filename) for filename in filenames]
    date_filename_pairs.sort(key=lambda x: x[0])
    sorted_filenames = [filename for _, filename in date_filename_pairs]
    return sorted_filenames

In [ ]:
    
    kelp_neighborhood = 5
    min_kelp_count = 4
    kelp_dilation_size = 15
    
    structuring_element = cp.ones((kelp_dilation_size, kelp_dilation_size))    

kelp_dilated_gpu = cp.where(classified_img_gpu == 0, True, False)  # This is expanding the kelp_mask so the TF is reversed
    kernel = cp.ones((kelp_neighborhood, kelp_neighborhood), dtype=cp.int32)
    
    time_val = time.time()
    kelp_count_gpu = convolve(kelp_dilated_gpu.astype(cp.int32), kernel, mode='constant', cval=0.0)
    print(f'kelp moving average finished: {time.time()-time_val}')

    kelp_dilated_gpu = cp.where(((~kelp_dilated_gpu) | (kelp_count_gpu <= min_kelp_count)), 0, 1)  # If there's no kelp, or the kelp count is <=4, set pixel == false
    time_val = time.time()
    kelp_dilated_gpu = binary_dilation(kelp_dilated_gpu, structure=structuring_element)

In [ ]:
from PIL import Image

image_files = os.listdir(r'/mnt/c/Users/attic/HLS_Kelp/plots')

images = [Image.open(os.path.join(r'/mnt/c/Users/attic/HLS_Kelp/plots', img)) for img in image_files]

images[0].save('timeseries.gif',
               save_all=True,
               append_images=images[1:],
               optimize=False,
               duration=250,  
               loop=0)   

In [ ]:
path = r'/mnt/c/Users/attic/HLS_Kelp/imagery/Isla_vista_kelp_2018/processed/11SKU'
files = os.listdir(path)
files = sort_filenames_by_date(files)

for i , file in enumerate(files):
    with rasterio.open(os.path.join(path,file)) as img:
        # r = img.read(3)
        # g = img.read(2)
        # b = img.read(1)
        mesma = img.read(6)
        kelp = img.read(5)
    # rgb_image = np.stack((r, g, b), axis=-1)

    # Normalize the image to 0-255 for display
    # rgb_image = np.clip(rgb_image, 0, 255)
    # rgb_image = rgb_image.astype(np.uint8)
    mesma = np.where(mesma <= 5, np.nan, mesma)

    plt.figure(figsize=(25, 25), dpi=100)
    # plt.imshow(rgb_image[2600:3200, 900:1600],alpha=1)
    plt.imshow(kelp[2600:3200, 900:1600], cmap = "Blues", alpha=.5)
    plt.imshow(mesma[2600:3200, 900:1600], vmin = 0, vmax = 100)

    plt.title(file)
    plt.axis('off')  # Hide axis
    plt.show()
    # plt.savefig(f'plot_{i+1}_v2.png')
    plt.close